```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [1]:
# NO TERMINAL
# pip install earthengine-api
# pip install geemap
# pip install pywebhdfs

In [2]:
# DOWNLOAD FROM GEE

In [8]:
import ee
import geemap
from pywebhdfs.webhdfs import PyWebHdfsClient
from datetime import date, timedelta
import os

In [4]:
# Inicia o fluxo de autenticação
ee.Authenticate()

# Inicializa a biblioteca
ee.Initialize()

Enter verification code:  4/1AX4XfWgChJ0zb8Moe0TBT-joyNm6DNKHo6L18a0TkCyKTSFgnOFsGSwn92A



Successfully saved authorization token.


In [5]:
# VARIAVEIS GLOBAIS (MUDAR AQUI!)
end_date = date.today()
percent_cloud = 100
# box = [[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin], [xmin, ymin]]
# https://boundingbox.klokantech.com/
box = [
    [
        [-54.6306579887, -25.5892766534],
        [-54.5393341362, -25.5892766534],
        [-54.5393341362, -25.4046299874],
        [-54.6306579887, -25.4046299874],
        [-54.6306579887, -25.5892766534],
    ]
]
boundary = ee.Geometry.Polygon(box, None, False)
collection_name = "COPERNICUS/S2_SR"
scale = 10
crs = "EPSG:4326"
# CRIE UMA PASTA VAZIA NO DIRETORIO RASTER E COLOQUE O CAMINHO DELA AQUI EM BAIXO
out_dir = "raster/sentinel2_tmp"
hdfs_dir = "sentinel2_tmp"

In [9]:
for i in range(2, 15, 2):
    start_date = str((end_date - timedelta(days=i)))
    collection = (
        ee.ImageCollection(collection_name)
        .select(["B2", "B3", "B4", "B8"])
        .filterBounds(boundary)
        .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", percent_cloud)
        .filterDate(start_date, str(end_date))
    )
    geemap.ee_export_image_collection(
        collection, scale=scale, crs=crs, region=boundary, out_dir=out_dir
    )
    #     NOT SURE WHAT F.. THIS BELOW DOES
    for root, directory, files in os.walk(out_dir):
        if len(files) == 0:
            geemap.ee_export_image_collection(
                collection, scale=scale, crs=crs, region=boundary, out_dir=out_dir
            )
        else:
            print("Existem imagens na pasta")

Total number of images: 0

Total number of images: 0

Total number of images: 1

Exporting 1/1: 20211226T134211_20211226T134212_T21JYM.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp/20211226T134211_20211226T134212_T21JYM.tif


Existem imagens na pasta
Total number of images: 1

Exporting 1/1: 20211226T134211_20211226T134212_T21JYM.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp/20211226T134211_20211226T134212_T21JYM.tif


Existem imagens na pasta
Total number of images: 1

Exporting 1/1: 20211226T134211_20211226T134212_T21JYM.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp/20211226T134211_20211226T134212_T21JYM.tif


Existem imagens na pasta
Total number of images: 2

Exporting 1/2: 20211221T134209_20211221T134205_T21JYM.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp/20211221T134209_20211221T134205_T21J

In [ ]:
hdfs = PyWebHdfsClient(host="179.106.229.159", port="50070", user_name="root")
hdfs.delete_file_dir(hdfs_dir, recursive=True)
hdfs.make_dir(hdfs_dir)

In [ ]:
# Adicionar imagens no HDFS

# No terminal do JupyterLab
# sudo docker exec -itu 0 hadoop passwd (primeira vez) - hoje é admin123
# ssh hadoop
# cd ..
# cd hadoop/bin
# ./hadoop fs -copyFromLocal /opt/workspace/raster/sentinel2_tmp/* /sentinel2_tmp